<a href="https://colab.research.google.com/github/blackwithwhitegreen/kaggle-template/blob/main/All_time_Best_performace_Analyze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
baddu01_ipl_2025_player_lifetime_statistics_path = kagglehub.dataset_download('baddu01/ipl-2025-player-lifetime-statistics')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/ipl-2025-player-lifetime-statistics/cricket_data_2025.csv',na_values = ['(NA)','(NAN)','(inf)']).fillna(0)

In [ ]:
df

In [ ]:
df = df.drop(columns=['Best_Bowling_Match'])

# DATA PREPROCESSING

In [ ]:
df = df.replace({'No stats':'0','NaN':'0'})

In [ ]:
df['Highest_Score'] = df['Highest_Score'].str.replace('*', '', regex=False)

# Convert 'Highest_Score' to numeric
df['Highest_Score'] = pd.to_numeric(df['Highest_Score'], errors='coerce')

In [ ]:
df

In [ ]:
df.columns.values

In [ ]:
df.info()

In [ ]:
cols = ['Matches_Batted', 'Not_Outs', 'Runs_Scored',
       'Highest_Score', 'Batting_Average', 'Balls_Faced',
       'Batting_Strike_Rate', 'Centuries', 'Half_Centuries', 'Fours',
       'Sixes', 'Catches_Taken', 'Stumpings', 'Matches_Bowled',
       'Balls_Bowled', 'Runs_Conceded', 'Wickets_Taken',
       'Bowling_Average', 'Economy_Rate',
       'Bowling_Strike_Rate', 'Four_Wicket_Hauls', 'Five_Wicket_Hauls']

In [ ]:
df[cols] = df[cols].apply(pd.to_numeric,errors='ignore')

In [ ]:
df[cols] = df[cols].astype('int',errors ='ignore')

In [ ]:
df.info()

In [ ]:
df

In [ ]:
# Replace all non-numeric values with 0 in the 'cols' columns
for col in cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

new_df = df.groupby('Player_Name')[cols].max()
new_df

In [ ]:
# Filter out rows where both 'Matches_Bowled' and 'Matches_Batted' are 0
df = df[(df['Matches_Bowled'] != 0) | (df['Matches_Batted'] != 0)]

for col in cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

new_df = df.groupby('Player_Name')[cols].max().reset_index()
new_df

In [ ]:
new_df.columns.values

# FULL VIZUALIZATION

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
for features in cols:
    if features in df.columns:
        plt.figure(figsize=(12,5))
        sns.lineplot(new_df[features].dropna(),marker='o',color='purple')
        plt.title(f'Histogram of {features}')
        plt.xlabel(features)
        plt.ylabel('Count')
        plt.tight_layout()
        plt.show()



for features in cols:
    if features in df.columns:
        plt.figure(figsize=(12,5))
        sns.histplot(new_df[features].dropna(),kde = True,color='green')
        plt.title(f'Histogram of {features}')
        plt.xlabel(features)
        plt.ylabel('Count')
        plt.tight_layout()
        plt.show()



cols_df = new_df.select_dtypes(include=[np.number])
if cols_df.shape[1] >= 4:
    plt.figure(figsize=(12,5))
    correlation_matrix = cols_df.corr()
    sns.heatmap(correlation_matrix,annot=True,cmap='coolwarm',fmt = '.2f')
    plt.title('Correlation Heatmap of Numeric Features.')
    plt.tight_layout()
    plt.show()




# PAIR PLOT FOR THE BATTING STATS
selected_columns = ['Matches_Batted', 'Not_Outs', 'Runs_Scored',
       'Highest_Score', 'Batting_Average',
       'Batting_Strike_Rate', 'Centuries', 'Half_Centuries', 'Fours', 'Sixes']
if set(selected_columns).issubset(new_df.columns):
    sns.pairplot(new_df[selected_columns].dropna(),diag_kind = 'kde')
    plt.show()



#PAIR PLOT FOR THE BOWLING STATS
selected_columns = ['Matches_Bowled', 'Balls_Bowled',
       'Runs_Conceded', 'Wickets_Taken', 'Bowling_Average', 'Economy_Rate',
       'Bowling_Strike_Rate', 'Four_Wicket_Hauls', 'Five_Wicket_Hauls']
if set(selected_columns).issubset(new_df.columns):
    sns.pairplot(new_df[selected_columns].dropna(),diag_kind = 'kde')
    plt.show()



# BATTING STATS FOR ALL TIME.
cateogrical_columns = ['Player_Name','Runs_Scored']

for features in cateogrical_columns:
    if features in new_df.columns:
        top_player = new_df.nlargest(20,'Runs_Scored')
        plt.figure(figsize=(20,5))
        sns.barplot(y ='Runs_Scored',x=features,data = top_player,palette='viridis', order=top_player[features].value_counts().index)
        plt.title(f'Count Plot of {features}')
        plt.tight_layout()
        plt.xticks(rotation=90)
        plt.show()



# BOWLING STATS FOR ALL TIME
cateogrical_columns = ['Player_Name','Wickets_Taken']

for features in cateogrical_columns:
    if features in new_df.columns:
        top_player = new_df.nlargest(20,'Wickets_Taken')
        plt.figure(figsize=(20,5))
        sns.barplot(y ='Matches_Bowled',x=features,data = top_player,palette='viridis', order=top_player[features].value_counts().index)
        plt.title(f'Count Plot of {features}')
        plt.tight_layout()
        plt.xticks(rotation=90)
        plt.show()




In [ ]:
new_df.columns

# MODELS SECTION

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,r2_score,accuracy_score,mean_absolute_error
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
cols = ['Player_Name', 'Matches_Batted', 'Not_Outs',
       'Highest_Score', 'Batting_Average', 'Balls_Faced',
       'Batting_Strike_Rate', 'Centuries', 'Half_Centuries', 'Fours', 'Sixes',
       'Catches_Taken', 'Stumpings', 'Matches_Bowled', 'Balls_Bowled',
       'Runs_Conceded', 'Wickets_Taken', 'Bowling_Average', 'Economy_Rate',
       'Bowling_Strike_Rate', 'Four_Wicket_Hauls', 'Five_Wicket_Hauls']

In [ ]:
X = df[cols]
y = df['Runs_Scored']

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

# RandomForestRegressor

In [ ]:
rf = RandomForestRegressor(n_estimators=100)

In [ ]:
rf.fit(X_train,y_train)

In [ ]:
y_pred_rf = rf.predict(X_test)

In [ ]:
rmse = mean_absolute_error(y_pred_rf,y_test)
r2_score = r2_score(y_pred_rf,y_test)

In [ ]:
print(f'rmse:',rmse)
print(f'r2-score :',r2_score)

# LogisticRegression

In [ ]:
lr = LogisticRegression()

In [ ]:
lr.fit(X_train,y_train)

In [ ]:
y_pred_lr = lr.predict(X_test)

In [ ]:
acc_lr = accuracy_score(y_pred_lr,y_test)
print(f'acc_lr:',acc_lr)

# DecisionTreeClassifier

In [ ]:
dt = DecisionTreeClassifier()

In [ ]:
dt.fit(X_train,y_train)

In [ ]:
y_pred_dt = dt.predict(X_test)

In [ ]:
acc_dt = accuracy_score(y_pred_dt,y_test)
print(f'acc_lr:',acc_dt)

# SVC (Support Vector Classifier)

In [ ]:
sv = SVC()

In [ ]:
sv.fit(X_train,y_train)

In [ ]:
y_pred_sv = sv.predict(X_test)

In [ ]:
acc_sv = accuracy_score(y_pred_sv,y_test)
print(f'acc_sv:',acc_sv)

# GradientBoostingClassifier

In [ ]:
gb = GradientBoostingClassifier()

In [ ]:
gb.fit(X_train,y_train)

In [ ]:
y_pred_gb = gb.predict(X_test)

In [ ]:
acc_gb = accuracy_score(y_pred_gb,y_test)
print(f'acc_gb:',acc_gb)

# RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier()

In [ ]:
rfc.fit(X_train,y_train)

In [ ]:
y_pred_rfc = rfc.predict(X_test)

In [ ]:
acc_rfc = accuracy_score(y_pred_rfc,y_test)
print(f'acc_rfc:',acc_rfc)

# KNeighborsClassifier

In [ ]:
kn = KNeighborsClassifier()

In [ ]:
kn.fit(X_train,y_train)

In [ ]:
y_pred_kn = kn.predict(X_test)

In [ ]:
acc_kn = accuracy_score(y_pred_kn,y_test)
print(f'acc_rfc:',acc_kn)

# CONCLUSION
**- NEED THE FINE-TUENING**

****- RandomForestRegressor give the good *r2_score* but *MAE* need to more fine-tune.****